# **Naive Bayes Model**

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message
            
def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)


In [4]:
data = DataFrame({'message': [], 'class': []})

In [5]:
data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))

In [6]:
data.head(2)

,message,class
emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam


In [8]:
data.shape

(3001, 2)

In [7]:
data.tail(2)

,message,class
emails/ham\02500.05b3496ce7bca306bed0805425ec8621,"\n\nHi,\n\n\n\nI think you need to give us a l...",ham
emails/ham\99999,,ham


In [9]:
data['class'].value_counts()

ham     2501
spam     500
Name: class, dtype: int64

In [11]:
# We will use a Countvectorizer to split up each message into it's list of word, and throw that into Naive Bayes classification. We will call fit() and get trained smap classifier

In [13]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform( data['message'].values)
counts

<3001x62964 sparse matrix of type '<class 'numpy.int64'>'
	with 429785 stored elements in Compressed Sparse Row format>

In [14]:
# Build Naive Bayes model
clasifier = MultinomialNB()

In [16]:
# Train the Naive Bayes model
targets = data['class'].values
clasifier.fit(counts, targets)

MultinomialNB()

In [17]:
# Predictions

example = ['Free Viagra now!!!', 'Hello Dilip, today is workding day in Pune']
example_counts = vectorizer.transform(example)
predictions = clasifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

In [18]:
# Predictions
example = ['Yesterday, it was raining heavily in Mumbai!',\
           'We are fortune 500 company, you won lottery']
example_counts = vectorizer.transform(example)
predictions = clasifier.predict(example_counts)
predictions

array(['ham', 'spam'], dtype='<U4')

In [19]:
# Predictions
example = ['Yesterday, it was raining heavily in Mumbai!', 'gift voucher']
example_counts = vectorizer.transform(example)
predictions = clasifier.predict(example_counts)
predictions

array(['ham', 'ham'], dtype='<U4')